<a href="https://colab.research.google.com/github/Manuel-Gomez-05/SenalesySistemas2/blob/main/parical_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Solucion_Parcial_2_SyS_Manuel_Gomez_FINAL_V4.ipynb"""

import os
import subprocess
import time
import re

# --- 1. Instalación de Dependencias (Modo Silencioso) ---
print("--- 🌑 Configurando Entorno & Librerías ---")
with open(os.devnull, 'w') as devnull:
    subprocess.run([
        "pip", "install", "streamlit", "numpy", "scipy", "matplotlib", "pandas",
        "soundfile", "pydub", "yt-dlp", "-q"
    ], check=True, stdout=devnull)
    subprocess.run(["apt-get", "install", "ffmpeg", "-y", "-qq"], check=True, stdout=devnull, stderr=devnull)
print("✅ Instalación completada.")

# --- 2. Directorios ---
if not os.path.exists("pages"):
    os.makedirs("pages")

# ==========================================
# ARCHIVO 1: PORTADA (PRESENTACION)
# ==========================================
with open("presentacion.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - SyS 2025-II",
    page_icon="📡",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("📡 Parcial 2: Señales y Sistemas 2025-II")
st.markdown("**Profesor:** Andrés Marino Álvarez Meza, Ph.D.")
st.markdown("### **Estudiante:** Manuel Alejandro Gómez")
st.divider()

col1, col2 = st.columns(2)

with col1:
    st.info("### 📻 Punto 1: Modulación AM")
    st.markdown("""
    * **Simulación DSB-SC** (Doble Banda Lateral).
    * **Filtrado Espectral (FFT):** Eliminación de réplicas en alta frecuencia.
    * Procesamiento de señales de audio (WAV/YouTube).
    """)

with col2:
    st.warning("### 🏎️ Punto 2: Sistemas Dinámicos")
    st.markdown("""
    * **Equivalencia:** $Masa-Resorte$ vs $Circuito RLC$.
    * **Parámetros:** Cálculos de $R, L, C$ equivalentes.
    * **Análisis Transitorio:** $t_{amortiguada}, t_{natural}, t_{no amrotiguada}$ para Escalón, Impulso y Rampa.
    * **Control:** Lazo Cerrado con Ganancia $K_p$.
    """)

st.markdown("---")
st.caption("👈 Seleccione un punto en el menú lateral. | Modo Oscuro Activado 🌑")
''')

# ==========================================
# ARCHIVO 2: PUNTO 1 (MODULACIÓN)
# ==========================================
with open("pages/punto1_modulacion.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
import yt_dlp
from pydub import AudioSegment
import os

# Estilo Oscuro para gráficas
plt.style.use('dark_background')

st.set_page_config(page_title="Punto 1: Modulación", layout="wide")

# --- Funciones ---
@st.cache_data(show_spinner=False)
def download_audio(url, fs=44100):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': '%(id)s.%(ext)s',
            'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav',}],
            'quiet': True, 'no_warnings': True
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = f"{info['id']}.wav"

        audio = AudioSegment.from_file(filename)
        audio = audio.set_channels(1).set_frame_rate(fs)
        # Cortar segmento representativo
        if len(audio) > 15000: audio = audio[10000:15000]
        elif len(audio) > 5000: audio = audio[:5000]

        data = np.array(audio.get_array_of_samples()).astype(np.float32)
        if np.max(np.abs(data)) > 0: data = data / np.max(np.abs(data))
        try: os.remove(filename)
        except: pass
        return fs, data
    except: return None, None

def plot_spectrum_dark(t, y, fs, title, color='#00FFFF'):
    N = len(y)
    Y = fftshift(fft(y))
    f = fftshift(fftfreq(N, 1/fs))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))

    # Tiempo
    ax1.plot(t, y, color=color, lw=0.8)
    ax1.set_title(f"{title} (Tiempo)", color='white')
    ax1.set_xlabel("s")
    ax1.grid(True, alpha=0.15)

    # Frecuencia
    mag = np.abs(Y)/N
    ax2.plot(f, mag, color='#FF00FF', lw=0.8) # Magenta
    ax2.set_title(f"{title} (Frecuencia)", color='white')
    ax2.set_xlabel("Hz")
    ax2.set_xlim([-fs/2, fs/2])
    ax2.grid(True, alpha=0.15)

    st.pyplot(fig)
    plt.close(fig)

# --- UI ---
st.title("🎹 1. Demodulación AM (DSB-SC)")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

with st.sidebar:
    st.header("🎛️ Configuración")
    src = st.radio("Fuente de Audio", ["Tono de Prueba", "YouTube"])
    fc = st.slider("Portadora (fc)", 5000, 15000, 10000, 1000)
    cut = st.slider("Corte Filtro LPF", 1000, 6000, 4000, 500)

fs = 44100
audio, t = None, None

if src == "YouTube":
    url = st.text_input("URL Video", "https://www.youtube.com/watch?v=kJQP7kiw5Fk")
    if url:
        with st.spinner("Descargando..."):
            _, audio = download_audio(url)
else:
    t_f = np.linspace(0, 2, 2*fs)
    audio = 0.5*np.cos(2*np.pi*440*t_f) + 0.3*np.cos(2*np.pi*800*t_f)
    audio *= np.hanning(len(audio))

if audio is not None:
    t = np.linspace(0, len(audio)/fs, len(audio))

    st.subheader("1. Señal Original $m(t)$")
    c1, c2 = st.columns([1,3])
    c1.audio(audio, sample_rate=fs)
    with c2: plot_spectrum_dark(t, audio, fs, "Mensaje", '#00FF00')

    st.subheader("2. Señal Modulada $r(t)$")
    carrier = np.cos(2*np.pi*fc*t)
    mod = audio * carrier
    plot_spectrum_dark(t, mod, fs, "Modulada", '#FFFF00')

    st.subheader("3. Demodulación (Mezclador)")
    mix = mod * carrier
    plot_spectrum_dark(t, mix, fs, "Salida Mixer", '#00FFFF')

    st.subheader("4. Filtrado Espectral (FFT)")
    Y = fft(mix)
    freqs = fftfreq(len(mix), 1/fs)
    mask = np.abs(freqs) < cut
    rec = ifft(Y * mask).real * 2

    st.success(f"Filtro ideal aplicado en frecuencia (Corte: {cut} Hz).")
    c3, c4 = st.columns([1,3])
    c3.audio(rec, sample_rate=fs)
    with c4: plot_spectrum_dark(t, rec, fs, "Recuperada", '#FF00FF')
''')

# ==========================================
# ARCHIVO 3: PUNTO 2 (SISTEMAS) - CON R,L,C
# ==========================================
with open("pages/punto2_sistemas.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

plt.style.use('dark_background') # Modo Oscuro Graficas

st.set_page_config(page_title="Punto 2: Sistemas", layout="wide")

st.title("🏎️ 2. Sistemas Dinámicos y Control")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

# --- Layout: Inputs a la Izquierda, Resultados a la Derecha ---
c_left, c_right = st.columns([1, 2.5])

with c_left:
    st.markdown("### ⚙️ Planta Mecánica")
    m = st.number_input("Masa $m$ (kg)", 0.1, 10.0, 1.0)
    k = st.number_input("Resorte $k$ (N/m)", 1.0, 200.0, 25.0)

    cc = 2*np.sqrt(k*m)
    st.caption(f"Amort. Crítico ($c_c$): {cc:.2f}")

    sel = st.selectbox("Preset Amortiguamiento", ["Subamortiguado", "Crítico", "Sobreamortiguado"])
    if "Sub" in sel: val_c = cc*0.3
    elif "Crítico" in sel: val_c = cc
    else: val_c = cc*2.0

    c = st.slider("Amortiguador $c$", 0.0, cc*4, val_c)

    st.divider()
    st.markdown("### 🎛️ Estrategia de Control")
    mode = st.radio("Configuración", ["Lazo Abierto", "Lazo Cerrado (P)"])
    Kp = 0.0
    if "Cerrado" in mode:
        Kp = st.slider("Ganancia $K_p$", 1.0, 100.0, 50.0)
        st.info(f"Rigidez Aumentada: $k_{{eq}} = {k+Kp:.1f}$")

# --- Cálculos ---
# 1. Función de Transferencia
den = [m, c, k + Kp] if "Cerrado" in mode else [m, c, k]
num = [Kp] if "Cerrado" in mode else [1]
sys = signal.TransferFunction(num, den)

# 2. Parámetros Dinámicos
wn = np.sqrt(den[2]/den[0])
zeta = den[1]/(2*np.sqrt(den[2]*den[0]))
wd = wn*np.sqrt(1-zeta**2) if zeta<1 else 0

# 3. Equivalentes Eléctricos (SOLO EN LAZO ABIERTO TIENE SENTIDO FISICO DIRECTO)
# Usamos los parámetros de la planta física (m, c, k) sin Kp
# Relaciones: LC = m/k, RC = m/c. Fijamos C = 100 uF.
C_elec = 100e-6
L_elec = m / (k * C_elec)
R_elec = m / (c * C_elec) if c > 0 else np.inf

# 4. Métricas Temporales (Simulación)
t_s, y_s = signal.step(sys, T=np.linspace(0, 15, 2000))
fv = y_s[-1] # Valor final

# Tiempo Establecimiento (2%)
ts_txt = "∞"
bounds = np.where((y_s < fv*0.98) | (y_s > fv*1.02))[0]
if len(bounds)>0: ts_txt = f"{t_s[bounds[-1]]:.3f} s"

# Tiempo Pico (Tp)
tp_txt = f"{t_s[np.argmax(y_s)]:.3f} s" if zeta < 1 else "N/A"

# Tiempo Levantamiento (Tr)
if zeta < 1: # 0 -> 100%
    crs = np.where(y_s >= fv)[0]
    tr_txt = f"{t_s[crs[0]]:.3f} s" if len(crs)>0 else "> Sim"
else: # 10% -> 90%
    i10 = np.argmax(y_s >= 0.1*fv)
    i90 = np.argmax(y_s >= 0.9*fv)
    tr_txt = f"{t_s[i90]-t_s[i10]:.3f} s"

# --- Visualización ---
with c_right:
    # SECCIÓN NUEVA: EQUIVALENTE ELÉCTRICO
    st.markdown("#### ⚡ Equivalente Circuito RLC (Serie-Paralelo)")
    st.caption("Valores para replicar la dinámica de la planta mecánica (Lazo Abierto) fijando $C = 100 \mu F$:")

    ce1, ce2, ce3 = st.columns(3)
    ce1.metric("Resistencia ($R$)", f"{R_elec:.2f} Ω")
    ce2.metric("Inductancia ($L$)", f"{L_elec:.4f} H")
    ce3.metric("Capacitor ($C$)", "100 µF")

    st.divider()

    # Métricas de Desempeño
    st.markdown("#### 📊 Análisis de Respuesta Transitoria")
    k1, k2, k3, k4 = st.columns(4)
    k1.metric("$\zeta$ (Amort.)", f"{zeta:.3f}")
    k2.metric("$\omega_n$ (Frec.)", f"{wn:.2f}")
    k3.metric("$t_s$ (Establec.)", ts_txt)
    k4.metric("$t_r$ (Levant.)", tr_txt)

    # Gráficas
    tab1, tab2, tab3 = st.tabs(["📈 Tiempo", "diagrama de Bode", "Polos"])

    with tab1:
        u_type = st.radio("Entrada:", ["Escalón", "Impulso", "Rampa"], horizontal=True)
        fig, ax = plt.subplots(figsize=(10, 3.5))
        t = np.linspace(0, 15, 1000)

        if "Escalón" in u_type:
            _, y = signal.step(sys, T=t)
            ax.plot(t, y, '#00FFFF', lw=2)
            ax.axhline(fv, color='white', ls='--', alpha=0.5)
        elif "Impulso" in u_type:
            _, y = signal.impulse(sys, T=t)
            ax.plot(t, y, '#FF00FF', lw=2)
        else: # Rampa
            _, y, _ = signal.lsim(sys, U=t, T=t)
            ax.plot(t, y, '#FFFF00', lw=2, label="Salida")
            ax.plot(t, t, 'white', ls=':', alpha=0.3, label="Entrada")
            ax.legend()

        ax.grid(True, alpha=0.15)
        ax.set_title(f"Respuesta a {u_type}", color='white')
        st.pyplot(fig)

    with tab2:
        w, mag, phase = signal.bode(sys)
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 4))
        ax1.semilogx(w, mag, '#00FF00')
        ax1.set_ylabel("Mag (dB)")
        ax1.grid(True, alpha=0.2)
        ax2.semilogx(w, phase, '#00FFFF')
        ax2.set_ylabel("Fase (deg)")
        ax2.grid(True, alpha=0.2)
        st.pyplot(fig)

    with tab3:
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.scatter(np.real(sys.poles), np.imag(sys.poles), c='red', marker='x', s=100, label='Polos')
        if len(sys.zeros)>0: ax.scatter(np.real(sys.zeros), np.imag(sys.zeros), c='blue', s=100, marker='o', facecolors='none')
        ax.axhline(0, color='white', alpha=0.3)
        ax.axvline(0, color='white', alpha=0.3)
        ax.set_title("Mapa de Polos y Ceros", color='white')
        ax.grid(True, ls=':', alpha=0.2)
        ax.legend()
        st.pyplot(fig)
''')

# --- 3. Ejecución ---
print("\n--- 🚀 Iniciando Streamlit (V4 Final) ---")

# Descarga robusta cloudflared
if not os.path.exists("cloudflared"):
    subprocess.run(["curl", "-L", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-o", "cloudflared"], check=True, stdout=subprocess.DEVNULL)
    subprocess.run(["chmod", "+x", "cloudflared"], check=True)

# Ejecutar Streamlit (Tema Oscuro Forzado)
proc_streamlit = subprocess.Popen(
    ["streamlit", "run", "presentacion.py", "--server.port", "8501", "--server.address", "0.0.0.0", "--theme.base", "dark"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

with open("tunnel.log", "w") as log_file:
    proc_tunnel = subprocess.Popen(
        ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
        stdout=log_file, stderr=subprocess.STDOUT
    )

print("⏳ Generando enlace público (10s)...")
time.sleep(10)

url_publica = None
try:
    with open("tunnel.log", "r") as f:
        content = f.read()
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
        if match: url_publica = match.group(0)
except: pass

if url_publica:
    print(f"\n✅ ¡TU APP MANUEL ESTÁ LISTA!:\n👉 {url_publica} 👈\n")
else:
    print("\n⚠️ URL no encontrada. Revisa el log.")
    with open("tunnel.log", "r") as f: print(f.read()[-300:])

try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    proc_streamlit.terminate()
    proc_tunnel.terminate()

<>:272: SyntaxWarning: invalid escape sequence '\m'
<>:272: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipython-input-509783351.py:272: SyntaxWarning: invalid escape sequence '\m'
  st.caption("Valores para replicar la dinámica de la planta mecánica (Lazo Abierto) fijando $C = 100 \mu F$:")


--- 🌑 Configurando Entorno & Librerías ---
✅ Instalación completada.

--- 🚀 Iniciando Streamlit (V4 Final) ---
⏳ Generando enlace público (10s)...

✅ ¡TU APP MANUEL ESTÁ LISTA!:
👉 https://captain-circumstances-newfoundland-picked.trycloudflare.com 👈

